# ***Stock Market Price Prediction***

In [1]:
# Import necessary libraries
import yfinance as yf
import pandas as pd

# Load stock data for a specific ticker (e.g., Apple)
ticker = 'AAPL'
data = yf.download(ticker, start='2015-01-01', end='2023-01-01')

# Display the first few rows of the dataset
print(data.head())


[*********************100%***********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2015-01-02  27.847500  27.860001  26.837500  27.332500  24.373962  212818400
2015-01-05  27.072500  27.162500  26.352501  26.562500  23.687302  257142000
2015-01-06  26.635000  26.857500  26.157499  26.565001  23.689539  263188400
2015-01-07  26.799999  27.049999  26.674999  26.937500  24.021719  160423600
2015-01-08  27.307501  28.037500  27.174999  27.972500  24.944685  237458000


In [2]:
# Preprocessing
# We will use only the 'Close' column for predicting future prices
data = data[['Close']]

# Checking for missing values
data.isnull().sum()

# Fill missing values
data = data.fillna(method='ffill')


<ipython-input-2-93f50f268fdf>:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


In [3]:
# Add a moving average (7-day and 21-day)
data['MA7'] = data['Close'].rolling(window=7).mean()
data['MA21'] = data['Close'].rolling(window=21).mean()

# Add date-related features (day, month, year)
data['Day'] = data.index.day
data['Month'] = data.index.month
data['Year'] = data.index.year

# Drop any remaining NaN values after feature engineering
data = data.dropna()


In [4]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Normalize data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

# Prepare data for LSTM model
def prepare_data(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

X, y = prepare_data(scaled_data)

# Reshape data for LSTM input
X = X.reshape((X.shape[0], X.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=10, batch_size=32)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


61/61 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - loss: 0.0498
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 6.3596e-04
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 7s 118ms/step - loss: 5.8952e-04
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 7s 107ms/step - loss: 5.9466e-04
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - loss: 5.4270e-04
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - loss: 5.4347e-04
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - loss: 5.1494e-04
Epoch 8/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 5.0688e-04
Epoch 9/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 4.9462e-04
Epoch 10/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - loss: 5.0282e-04


In [5]:
import plotly.graph_objects as go

# Line chart for stock prices
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Close Price'))
fig.add_trace(go.Scatter(x=data.index, y=data['MA7'], mode='lines', name='7-Day MA'))
fig.add_trace(go.Scatter(x=data.index, y=data['MA21'], mode='lines', name='21-Day MA'))

fig.update_layout(title='Stock Price with Moving Averages', xaxis_title='Date', yaxis_title='Price', template='plotly_dark',  # Set the dark theme
    showlegend=True)
fig.show()


In [9]:
pip install dash dash_bootstrap_components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 3.6 MB/s eta 0:00:00


In [10]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import yfinance as yf
import plotly.graph_objects as go

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

# Layout of the dashboard
app.layout = dbc.Container([
    dbc.Row(html.H1('Stock Price Prediction Dashboard', style={'textAlign': 'center'})),
    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id='stock-picker',
            options=[{'label': ticker, 'value': ticker} for ticker in ['AAPL', 'GOOG', 'TSLA']],
            value='AAPL',
            style={'color': 'black'}
        ), width=4),
        dbc.Col(dcc.Slider(
            id='year-slider',
            min=2015,
            max=2023,
            step=1,
            value=2020,
            marks={i: str(i) for i in range(2015, 2024)}
        ), width=10),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='line-chart'), width=12)
    ]),
    html.Div('Created by AI Innovators', style={'textAlign': 'center', 'marginTop': '20px'})
])

# Callback for updating graph based on user input
@app.callback(
    Output('line-chart', 'figure'),
    [Input('stock-picker', 'value'),
     Input('year-slider', 'value')]
)
def update_graph(stock, year):
    # Load data based on stock and year
    data = yf.download(stock, start=f'{year}-01-01', end=f'{year}-12-31')
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Close Price'))

    # Update layout for dark theme
    fig.update_layout(
        template='plotly_dark',
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font=dict(color='white')
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

# **HAPPY COADING**